In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv

In [2]:
# Initial imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [3]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [4]:
## Import symbol list 
path = Path('../Resources/list_TI_export_success.pkl')
key_list = load_obj(path)
len(key_list)

932

In [5]:
test_symbol = key_list[10]
symbol = 'GME'
symbol

'GME'

In [6]:
## Get data 
path = Path('../FilesExport_DFs_with_TI_pkl/'+symbol+'_data_dict_with_technicals.pkl')
data_import = load_obj(path)

df = data_import[symbol].copy()

## Set X and y data 
X = df.drop(columns={'close','adjClose'}).values
y = df['close'].values

In [7]:
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
print(len(X_train_scaled))
print(len(X_test_scaled))

1013
435


In [12]:
validation_split_value=0.3
epochs_value=400
units1 = 8
units2 = 8

num_of_inputs = 20
num_of_outputs = 1

In [13]:
nn = Sequential()

# First hidden layer
nn.add(Dense(units=units1, input_dim=num_of_inputs, activation="relu"))

# Second hidden layer
nn.add(Dense(units=units2, activation="relu"))

# Output layer
nn.add(Dense(units=num_of_outputs, activation="linear"))

# Compile the model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

# Fit the model
model = nn.fit(X, y, 
                  validation_split=validation_split_value, 
                  epochs=epochs_value, 
                  verbose=0)

In [14]:
model_loss, model_accuracy = nn.evaluate(X, y, verbose=0)

In [15]:
print(model_loss)
print(model_accuracy)

9.006733894348145
9.006733894348145


In [16]:
score = nn.evaluate(X, y, verbose=0)
score

[9.006733894348145, 9.006733894348145]

In [19]:
df_pred = nn.predict(X_test_scaled)
#df_pred

In [20]:
# Save model as JSON
nn_json = nn.to_json()
file_path = Path("../Resources/tempmodel06.json")
with open(file_path, "w") as json_file:
    json_file.write(nn_json)

# Save weights
file_path = "../Resources/tempmodel06.h5"
nn.save_weights(file_path)

# Load json and create model
file_path = Path("../Resources/tempmodel06.json")
with open(file_path, "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

# Load weights into new model
file_path = "../Resources/tempmodel06.h5"
loaded_model.load_weights(file_path)

In [24]:
df2 = df.copy()
#df2 = df2.tail(len(X_test_scaled))
df2['predicted'] = loaded_model.predict(X)
df_pred = df2[['close','predicted']]
df_pred

,close,predicted
Date,,
2016-02-02,25.920000,25.853632
2016-02-03,26.170000,25.736700
2016-02-04,27.010000,26.871035
2016-02-05,26.830000,26.831852
2016-02-08,27.889999,27.966551
...,...,...
2021-10-25,173.970000,173.460480
2021-10-26,177.840000,179.797424
2021-10-27,173.510000,174.178894
